# Guide01

## Introduction

This guide explains how to obtain SpatialIDs within a certain distance of a given line. We'll review the following functions:
- GetSpatialIdsWithinRadiusOfLine()
- GetNspatialIdsAroundVoxcels()
- FitClearanceAroundExtendedSpatialID()

### External Data and License Notice
This guide will use geodetic coordinates from [OpenStreetMap](openstreetmap.org/copyright); please note that the data is available under the Open Database License. Please refer to the link for more information. The following code is not part of OpenStreeMap; it's provided by Trajectory under the MIT license. Please feel free to learn and contribute!

### About Jupyter Notebooks with Go

This notebook uses [GoNB](https://github.com/janpfeifer/gonb) for the Go integration. Please see the repo for instruction on how to set it up so you can run this notebook yourself.

As explained [here](https://github.com/janpfeifer/gonb/blob/main/examples/tutorial.ipynb), the `%%` shortcut is a feature of GoNB. It wraps subsequent code in a main() function that allows for things like a `fmt.Print()` call. If you're copying the code into a standard `.go` file, please exclude `%%`.

In [37]:
// setup cell: import packages
import (
	"fmt"
	"math"
	"github.com/go-gl/mathgl/mgl64"
    "github.com/trajectoryjp/geodesy_go/coordinates"
	closest "github.com/trajectoryjp/closest_go"
	geodesy "github.com/trajectoryjp/geodesy_go/coordinates"
	"github.com/trajectoryjp/spatial_id_go/v2/common/errors"
	"github.com/trajectoryjp/spatial_id_go/v2/common/object"
	"github.com/trajectoryjp/spatial_id_go/v2/shape"
	transform "github.com/trajectoryjp/spatial_id_go/v2/transform"
	"github.com/janpfeifer/gonb/cache"
)

%%
fmt.Print("Setup complete")


Setup complete

## Part 1: Simple Use Case: GetSpatialIdsWithinRadiusOfLine()

It's sakura season here in Japan -- which means there's no better time to learn about the inner workings of Spatial IDs! Let's go to a popular [hanami](https://en.wikipedia.org/wiki/Hanami) spot in Tokyo: [Yoyogi Park](https://en.wikipedia.org/wiki/Yoyogi_Park).

Bringing a baseball is never a bad idea. But it's crowded.



### Parameters

Let's find a straight-line area where there aren't any people and find the Spatial IDs within 10 meters of that line. Let's take a look at the parameters we need to run `GetSpatialIdsWithinRadiusOfLine()`:
- A startPoint in `*object.Point` format.
- An endPoint in `*object.Point` format.
- radius distance from the line as `float64`. We have our line from the dog park to the tree, but we want a "buffer" for throwing our baseball around this line. We're basically constructing a cylinder lengthwise; the radius parameter tells us how wide the cylinder is.
- hZoom, or Horizontal Zoom as `int64`.
- vZoom, or Vertical Zoom, as `int64`.
- a boolean value indicating if we want to skip the measurement of each Spatial ID to the straight line. Indicating `false` will measure the closest distance of all returned voxels and decide, with micrometer precision, if the voxcel is within the radius distance of the line. `false` results in a slow, precision-priority function. `true` will skip this measurement routine and return Spatial ID voxels that are within radius distance, but will over-estimate: voxels within radius distance + at most 1 voxcel-distance margin of error will be returned. `false` is faster but will result in a larger prizm of resulting voxels that indicated by radius.


Let's use the following points and parameters. PointA is by the dog park and pointB is under a tree.
- startPoint = [longitude: 139.69441 degrees, latitude: 35.67269 degrees, altitude (from sea-level): 40 meters ]
- endPoint = [longitude: 139.69516 degrees, latitude: 35.67157 degrees, altitude (from sea-level): 40 meters ]
- radius = 10 meters
- hZoom = level 25 (resutling voxel x-y distance of about 1.2 meters)
- vZoom = level 25 (resulting voxcel height of about 1 meter) 
- skipsMeasurement = false (for now!)

Note: See the [Spatial ID Guide](spatialID.md) for more detailed information on Zoom Levels

### Code 
First we declare our points as `*object.Point`, and then run `GetSpatialIdsWithinRadiusOfLine()`

In [38]:
var (
	pointA *object.Point
	pointB *object.Point
) 


%%
pointA, error := object.NewPoint(139.69441, 35.67269, 40)
if error != nil {
	fmt.Print(error)
	}
pointB, error := object.NewPoint(139.69516, 35.67157, 40)
if error != nil {
	fmt.Print(error)
}


fmt.Printf("pointA: %v\npointB: %v", *pointA, *pointB)

idsWithinRadiusOfLine, error := transform.GetSpatialIdsWithinRadiusOfLine(
	pointA, // startPoint
	pointB, // endPoint
	10,     // radius (in meters)
	25,     // hZoom
	25,     // vZoom
	false,   // skipsMeasurement
)
if error != nil {
	fmt.Print(error)
}

fmt.Printf("\nWe found %v Spatial IDs!\nExtended Spatial IDs within 10 meters of the line between pointA and pointB above:\n", len(idsWithinRadiusOfLine), idsWithinRadiusOfLine)


pointA: {139.69441 35.67269 40}
pointB: {139.69516 35.67157 40}
We found 82547 Spatial IDs!
Extended Spatial IDs within 10 meters of the line between pointA and pointB above:
%!(EXTRA []string=[25/29797705/13214035/25/47 25/29797707/13214045/25/34 25/29797745/13214092/25/47 25/29797677/13213977/25/39 25/29797720/13214069/25/33 25/29797736/13214093/25/38 25/29797709/13214049/25/30 25/29797750/13214097/25/40 25/29797691/13213978/25/48 25/29797700/13214024/25/43 25/29797705/13214033/25/33 25/29797673/13213970/25/49 25/29797691/13214003/25/37 25/29797702/13214010/25/39 25/29797719/13214064/25/32 25/29797746/13214097/25/32 25/29797747/13214093/25/43 25/29797705/13214015/25/37 25/29797707/13214023/25/37 25/29797699/13214008/25/51 25/29797739/13214101/25/45 25/29797679/13213985/25/33 25/29797733/13214060/25/40 25/29797683/13213978/25/32 25/29797740/13214073/25/37 25/29797729/13214093/25/36 25/29797695/13214007/25/47 25/29797746/13214112/25/51 25/29797729/13214073/25/37 25/29797744/13214112/25

## Part 2: Speed Tests

All parameters affect the speed of `GetSpatialIdsWithinRadiusOfLine()`. Conceptually, the calculation time increases:
- As the distance between the two points increases
- As the radius increases
- As the size of the voxels decreases (or as zoom levels increase)
- if `skipsMeasurement` is false


### Comparing `skipsMeasurement`
Let's time the previous test, where the distance between points A and B is about 100 meters.

In [39]:
func Benchmark_measure(b *testing.B) {

	pointA, error := object.NewPoint(139.69441, 35.67269, 40)
	if error != nil {
		b.Fatal(error)
	}
	pointB, error := object.NewPoint(139.69516, 35.67157, 40)
	if error != nil {
		b.Fatal(error)
	}
	idsWithinRadiusOfLine, error := transform.GetSpatialIdsWithinRadiusOfLine(
		pointA,
		pointB,
		10,     // radius (in meters)
		25,     // hZoom
		25,     // vZoom
		false,   // skipsMeasurement
	)
	if error != nil {
		b.Fatal(error)
	}
	fmt.Printf("Number of Ids returned: %v\n", len(idsWithinRadiusOfLine))
}
%test

goos: darwin
goarch: amd64
pkg: gonb_890a45fb
cpu: Intel(R) Core(TM) i5-10600 CPU @ 3.30GHz
Benchmark1
82547Benchmark1-12                	       1	2445832208 ns/op
Benchmark_measure
Number of Ids returned: 82547
Benchmark_measure-12         	       1	2226195281 ns/op
Benchmark_skipsMeasure
Number of Ids returned: 117438
Benchmark_skipsMeasure-12    	       1	1580280230 ns/op
PASS


Now let's benchmark the same test, but without precision measurement. Now `skipsMeasurement=true`

In [46]:
func Benchmark_skipsMeasure(b *testing.B) {

	pointA, error := object.NewPoint(139.69441, 35.67269, 40)
	if error != nil {
		b.Fatal(error)
	}
	pointB, error := object.NewPoint(139.69516, 35.67157, 40)
	if error != nil {
		b.Fatal(error)
	}
	idsWithinRadiusOfLine, error := transform.GetSpatialIdsWithinRadiusOfLine(
		pointA,
		pointB,
		10,     // radius (in meters)
		25,     // hZoom
		25,     // vZoom
		true,   // skipsMeasurement
	)
	if error != nil {
		b.Fatal(error)
	}
	fmt.Printf("Number of Ids returned: %v\n", len(idsWithinRadiusOfLine))
}
%test -test.bench=Benchmark_skipsMeasure -test.run=Bechmark

Number of Ids returned: 127650
goos: darwin
goarch: amd64
pkg: gonb_890a45fb
cpu: Intel(R) Core(TM) i5-10600 CPU @ 3.30GHz
Benchmark_skipsMeasure-12    	       1	1824789968 ns/op
PASS


With `skipsMeasure=true`, the function returns more Spatial IDs, but does so in about 2/3 the time.

### Comparing Radius
Let's reduce the radius in half to 5 meters and compare


In [44]:
func Benchmark_radius(b *testing.B) {

	pointA, error := object.NewPoint(139.69441, 35.67269, 40)
	if error != nil {
		b.Fatal(error)
	}
	pointB, error := object.NewPoint(139.69516, 35.67157, 40)
	if error != nil {
		b.Fatal(error)
	}
	idsWithinRadiusOfLine, error := transform.GetSpatialIdsWithinRadiusOfLine(
		pointA,
		pointB,
		5,     // radius (in meters)
		25,     // hZoom
		25,     // vZoom
		false,   // skipsMeasurement
	)
	if error != nil {
		b.Fatal(error)
	}
	fmt.Printf("Number of Ids returned: %v\n", len(idsWithinRadiusOfLine))
}
%test -test.bench=Benchmark_radius -test.run=Bechmark

Number of Ids returned: 23569
goos: darwin
goarch: amd64
pkg: gonb_890a45fb
cpu: Intel(R) Core(TM) i5-10600 CPU @ 3.30GHz
Benchmark_radius-12    	Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
1000000000	         0.4689 ns/op
PASS


### Comparing Zoom Level
Even if we decrease the Zoom Level by just one, the speed increases noticeably. Zoom Level 24 results in [about a 2.4m x 2.4m x 2m voxel](https://www.ipa.go.jp/digital/architecture/Individual-link/nq6ept000000g0fh-att/4dspatio-temporal-guideline-gamma.pdf)

In [42]:
func Benchmark_zoomLevel(b *testing.B) {

	pointA, error := object.NewPoint(139.69441, 35.67269, 40)
	if error != nil {
		b.Fatal(error)
	}
	pointB, error := object.NewPoint(139.69516, 35.67157, 40)
	if error != nil {
		b.Fatal(error)
	}
	idsWithinRadiusOfLine, error := transform.GetSpatialIdsWithinRadiusOfLine(
		pointA,
		pointB,
		10,     // radius (in meters)
		24,     // hZoom
		24,     // vZoom
		false,   // skipsMeasurement
	)
	if error != nil {
		b.Fatal(error)
	}
	fmt.Printf("Number of Ids returned: %v\n", len(idsWithinRadiusOfLine))
}
%test

goos: darwin
goarch: amd64
pkg: gonb_890a45fb
cpu: Intel(R) Core(TM) i5-10600 CPU @ 3.30GHz
Benchmark1
82547Benchmark1-12                	       1	2304384105 ns/op
Benchmark_measure
Number of Ids returned: 82547
Benchmark_measure-12         	       1	2243944878 ns/op
Benchmark_radius
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 27195
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 27195
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 27195
Number of Ids returned: 2